In [2]:
import os
if not os.path.exists('nb2_figs/tut_batch_grid'):
    os.makedirs('nb2_figs/tut_batch_grid')
os.chdir(os.getcwd() + '/nb2_figs/tut_batch_grid')
print(os.getcwd())

/mnt/disk15tb/adam/git_workspace/netpyne/hdmea_simulations/BurstingPlotDevelopment/nb2_figs/tut_batch_grid


In [ ]:
#imports and functions
from netpyne import specs, sim


In [ ]:
#Define Parameter Space
## Network parameters

#Population Parameter held constant
netParams = specs.NetParams()  # object of class NetParams to store the network parameters

netParams.sizeX = 4000              # x-dimension (horizontal length) size in um
netParams.sizeY = 2000             # y-dimension (vertical height or cortical depth) size in um
netParams.sizeZ = 0              # z-dimension (horizontal length) size in um

## Population parameters
netParams.popParams['E'] = {
    'cellType': 'E', 
    'numCells': 200, 
    'yRange': [100,1900], 
    'xRange': [100,3900]}
netParams.popParams['I'] = {
    'cellType': 'I', 
    'numCells': 200, 
    'yRange': [100,1900], 
    'xRange': [100,3900]}

# Simulation options
cfg = specs.SimConfig()     # object of class SimConfig to store simulation configuration

seconds = 1
cfg.duration = seconds*1e3        # Duration of the simulation, in ms
cfg.dt = 0.025              # Internal integration timestep to use
cfg.verbose = False         # Show detailed messages
cfg.recordTraces = {'V_soma':{'sec':'soma','loc':0.5,'var':'v'}}  # Dict with traces to record
cfg.recordStep = 0.1        # Step size in ms to save data (eg. V traces, LFP, etc)
cfg.filename = 'tut8'       # Set file output name
cfg.saveJson = True
cfg.printPopAvgRates = True
cfg.analysis['plotRaster'] = {'saveFig': True}                   # Plot a raster
cfg.analysis['plotTraces'] = {'include': [0], 'saveFig': True}  # Plot recorded traces for this list of cells

cfg.saveDataInclude = ['simData', 'simConfig', 'netParams', 'net']

# Variable parameters (used in netParams)
cfg.synMechTau2 = 5
cfg.connWeight = 0.01

# Network parameters
netParams = specs.NetParams()  # object of class NetParams to store the network parameters

## Cell parameters/rules
PYRcell = {'secs': {}}
PYRcell['secs']['soma'] = {'geom': {}, 'mechs': {}}  # soma params dict
PYRcell['secs']['soma']['geom'] = {'diam': 18.8, 'L': 18.8, 'Ra': 123.0}  # soma geometry
PYRcell['secs']['soma']['mechs']['hh'] = {'gnabar': 0.12, 'gkbar': 0.036, 'gl': 0.003, 'el': -70}  # soma hh mechanism
netParams.cellParams['PYR'] = PYRcell

## Population parameters
netParams.popParams['S'] = {'cellType': 'PYR', 'numCells': 20}
netParams.popParams['M'] = {'cellType': 'PYR', 'numCells': 20}

## Synaptic mechanism parameters
netParams.synMechParams['exc'] = {'mod': 'Exp2Syn', 'tau1': 0.1, 'tau2': cfg.synMechTau2, 'e': 0}  # excitatory synaptic mechanism

# Stimulation parameters
netParams.stimSourceParams['bkg'] = {'type': 'NetStim', 'rate': 10, 'noise': 0.5}
netParams.stimTargetParams['bkg->PYR'] = {'source': 'bkg', 'conds': {'cellType': 'PYR'}, 'weight': 0.01, 'delay': 5, 'synMech': 'exc'}

## Cell connectivity rules
netParams.connParams['S->M'] = {    #  S -> M label
    'preConds': {'pop': 'S'},       # conditions of presyn cells
    'postConds': {'pop': 'M'},      # conditions of postsyn cells
    'probability': 0.5,             # probability of connection
    'weight': cfg.connWeight,       # synaptic weight
    'delay': 5,                     # transmission delay (ms)
    'synMech': 'exc'}               # synaptic mechanism


In [ ]:
# #Define Parameter Space
# from netpyne import specs

# # Create variable of type ordered dictionary (NetPyNE's customized version)
# params = specs.ODict()

# # fill in with parameters to explore and range of values (key has to coincide with a variable in simConfig)
# params['synMechTau2'] = [3.0, 5.0, 7.0, 12]
# params['connWeight'] = [0.005, 0.01, 0.15, 0.20]

# Create variable of type ordered dictionary (NetPyNE's customized version)
params = specs.ODict()

# fill in with parameters to explore and range of values (key has to coincide with a variable in simConfig)
params['synMechTau2'] = list(range(1, 21))  # range from 1 to 20
params['connWeight'] = [x / 1000.0 for x in range(1, 1001)]  # range from 0.001 to 1
netParams.propVelocity = list(range(1, 1001))  # range from 0.1 to 1000 um/ms
netParams.probLengthConst = list(range(50, 501))  # range from 50 to 500 um